## 데이터 표준화를 풀고 파생변수를 생성할때 이용해보자!
- 현재 데이터는 원본값을 표준편차로 나누어서 변환됨
- 표준편차를 추정해서 매스킹을 풀어보자
- 다시 데이터로 쓸땐 min max scale?

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv("train_df.csv")

## playtime으로 연습

In [51]:
playtime_unique = train_df['playtime'].unique()     # 편차를 확인해보기 위함이므로 unique 처리를 해준다
playtime_unique_sorted = np.sort(playtime_unique)   # 편차확인위해 순서대로 sort 해줌

In [52]:
playtime_unique_sorted ## 잘 보면 배로 커진다... 2.37의 배수 

array([2.37012087e-03, 4.74024174e-03, 7.11036260e-03, ...,
       3.40823381e+00, 3.41060393e+00, 3.41297405e+00])

In [53]:
pd.DataFrame(pd.DataFrame(np.diff(playtime_unique_sorted))[0].value_counts()).reset_index()['index'][0]

0.0023701208675026564

In [54]:
## 내가 산출한 표준편차
sd = 1/0.0023406552170048833
sd

427.23079962182817

In [55]:
playtime_unique_sorted

array([2.37012087e-03, 4.74024174e-03, 7.11036260e-03, ...,
       3.40823381e+00, 3.41060393e+00, 3.41297405e+00])

In [56]:
playtime_unique_sorted*sd ## 정수화 됨

array([1.01258863e+00, 2.02517727e+00, 3.03776590e+00, ...,
       1.45610245e+03, 1.45711504e+03, 1.45812763e+03])

## 함수화

In [138]:
def unstandardize(train_df,variable):
    unique = train_df[variable].unique()
    unique_sorted = np.sort(unique)
    sd = 1/pd.DataFrame(pd.DataFrame(np.diff(unique_sorted))[0].value_counts()).reset_index()['index'][0]
    return sd,unique_sorted,train_df[variable]*sd

In [89]:
print(unstandardize(train_df,'playtime')[0],unstandardize(train_df,'playtime')[1].head()) ## 정수화됨

[2.37012087e-03 4.74024174e-03 7.11036260e-03 ... 3.40823381e+00
 3.41060393e+00 3.41297405e+00] 0    1155.0
1    1437.0
2    1440.0
3    1440.0
4    1440.0
Name: playtime, dtype: float64


In [56]:
for i in range(len(train_df.columns)):
    try:
        print("***" + train_df.columns[i] + "***")
        print("-----------------------------------")
    
        print(unstandardize(train_df,train_df.columns[i])[0])
        print("-----------------------------------")
    
        print(unstandardize(train_df,train_df.columns[i])[1][0:20])
        print("-----------------------------------")
    
        print(unstandardize(train_df,train_df.columns[i])[2].head())
    
    except TypeError:
        print("TypeError : " + train_df.columns[i])
    except KeyError as e1:
        print("KeyError : "  + train_df.columns[i])
    except IndexError:
        print("IndexError : " + train_df.columns[i])

***Unnamed: 0***
-----------------------------------
1.0
-----------------------------------
[44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
-----------------------------------
0    44.0
1    45.0
2    46.0
3    47.0
4    48.0
Name: Unnamed: 0, dtype: float64
***day***
-----------------------------------
1.0
-----------------------------------
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
-----------------------------------
0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
Name: day, dtype: float64
***acc_id***
-----------------------------------
1.0
-----------------------------------
[ 2  5  8 17 20 21 31 38 41 43 50 53 54 59 62 63 65 66 67 69]
-----------------------------------
0    2.0
1    2.0
2    2.0
3    2.0
4    2.0
Name: acc_id, dtype: float64
***char_id***
-----------------------------------
1.0
-----------------------------------
[ 2  3  5  7  9 13 14 15 17 19 22 23 26 28 29 31 32 34 37 41]
-----------------------------------
0    233598.0
1    233

10694.611196494972
-----------------------------------
[0.00000000e+00 9.16349348e-06 1.87010071e-05 8.41545320e-05
 1.12206043e-04 1.40257553e-04 1.87010071e-04 2.80421602e-04
 2.80515107e-04 2.81450157e-04 3.16608050e-04 3.27267624e-04
 3.74020142e-04 4.66590127e-04 4.67431673e-04 4.67525178e-04
 5.14651716e-04 5.61030213e-04 5.61965264e-04 6.17133235e-04]
-----------------------------------
0    NaN
1    0.0
2    NaN
3    NaN
4    NaN
Name: target_accessory_price, dtype: float64
***target_adena_price***
-----------------------------------
IndexError : target_adena_price
***target_armor_price***
-----------------------------------
3564870.3988373014
-----------------------------------
[0.00000000e+00 9.35050355e-08 9.35050355e-06 4.67525178e-05
 6.54535249e-05 7.66741291e-05 9.35050355e-05 9.35985406e-05
 9.81802873e-05 1.11270992e-04 1.40257553e-04 1.40912089e-04
 1.43623735e-04 1.70833700e-04 1.87010071e-04 1.96360575e-04
 2.15061582e-04 2.23570540e-04 2.33762589e-04 2.34884649e-04

[0.00000000e+00 9.35050355e-08 1.87010071e-07 3.74020142e-07
 9.35050355e-07 1.12206043e-06 1.30907050e-06 1.49608057e-06
 1.87010071e-06 2.05711078e-06 2.80515107e-06 3.08566617e-06
 3.55319135e-06 3.74020142e-06 4.67525178e-06 5.14277695e-06
 5.23628199e-06 5.61030213e-06 6.17133235e-06 1.03790589e-05]
-----------------------------------
0    0.0
1    0.0
2    0.0
3    NaN
4    0.0
Name: source_enchant_scroll_price, dtype: float64
***source_etc_price***
-----------------------------------
3564870.3988373014
-----------------------------------
[0.00000000e+00 9.35050355e-08 1.87010071e-07 3.74020142e-07
 4.67525178e-07 9.35050355e-07 1.02855539e-06 1.87010071e-06
 1.96360575e-06 2.61814100e-06 3.27267624e-06 3.36618128e-06
 3.74020142e-06 4.67525178e-06 4.95576688e-06 5.23628199e-06
 5.61030213e-06 5.79731220e-06 6.35834242e-06 6.45184745e-06]
-----------------------------------
0    1.210031e+07
1    7.068867e+05
2    5.363227e+06
3             NaN
4    0.000000e+00
Name: source_etc_

13.979886528454516
-----------------------------------
[0.         0.07153134 0.14306268 0.21459402 0.28612536 0.35765669
 0.42918803 0.50071937 0.57225071 0.64378205 0.71531339 0.78684473
 0.85837607 0.9299074  1.00143874 1.07297008 1.14450142 1.21603276
 1.2875641  1.35909544]
-----------------------------------
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: pledge_random_defender_cnt, dtype: float64
***pledge_same_pledge_cnt***
-----------------------------------
2.513455786326567
-----------------------------------
[0.        0.3978586 0.7957172 1.1935758 1.5914344 1.989293  2.3871516
 2.7850102 3.1828688 3.5807274 3.978586  4.3764446 4.7743032 5.1721618
 5.5700204 5.967879  6.3657376 6.7635962 7.1614548 7.5593134]
-----------------------------------
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: pledge_same_pledge_cnt, dtype: float64
***pledge_temp_cnt***
-----------------------------------
10.112956893026828
-----------------------------------
[0.         0.09888305 0.1977661  0.29